Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Distributed MPMLA using custom docker images
This recipe shows how to run High Performance ML Algorithms (HPMLA) on CPUs across Azure VMs via Open MPI.

## Prerequisites
* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning
* Go through the [00.configuration.ipynb](./00.configuration.ipynb) notebook to:
    * install the AML SDK
    * create a workspace and its configuration file (`config.json`)

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [ ]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics = True)

## Initialize workspace

Initialize a [Workspace](https://review.docs.microsoft.com/en-us/azure/machine-learning/service/concept-azure-machine-learning-architecture?branch=release-ignite-aml#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

## Create a remote compute target
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) to execute your training script on. In this tutorial, you create an [Azure Batch AI](https://docs.microsoft.com/azure/batch-ai/overview) cluster as your training compute resource. This code creates a cluster for you if it does not already exist in your workspace.

**Creation of the cluster may take up to a few minutes depending on `cluster_min_nodes`.** If the cluster is already in your workspace this code will skip the cluster creation process.

In [ ]:
from azureml.core.compute import ComputeTarget, BatchAiCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = BatchAiCompute.provisioning_configuration(vm_size='STANDARD_D2_v2', 
                                                                autoscale_enabled=True,
                                                                cluster_min_nodes=0, 
                                                                cluster_max_nodes=2)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

    # Use the 'status' property to get a detailed status for the current cluster. 
    print(compute_target.status.serialize())

## Upload training data
To make data accessible for remote training, AML provides a convenient way to do so via a [Datastore](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data). The datastore provides a mechanism for you to upload/download data to Azure Storage, and interact with it from your remote compute targets. 

For this tutorial, we will demonstrate how to upload the training data to a datastore and access it during training to illustrate the datastore functionality.

We first download the training data to your local machine. We start with a binary classification datasets in [here](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/). The biggest dataset would be [criteo_tb](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary.html#criteo_tb)

In [ ]:
import os
import urllib

os.makedirs('./data', exist_ok=True)
download_url = 'https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a9a'
urllib.request.urlretrieve(download_url, filename='./data/a9a')

Each workspace is associated with a default datastore. In this tutorial, we will upload the training data to this default datastore. The below code will upload the contents of the data directory to the path `./data` on the AML default blob datastore.

In [ ]:
from azureml.core import Datastore

ds = Datastore(ws, 'workspaceblobstore')
print(ds.datastore_type, ds.account_name, ds.container_name)
ds.upload(src_dir='data', target_path='HPMLA/rawdata', overwrite=True, show_progress=True)

Now let's get a reference to the path on the datastore with the training data. We can do so using the `path` method. In the next section, we can then pass this reference to our training script's corresponding arguments. 

In [ ]:
path_on_datastore = 'HPMLA'
ds_data = ds.path(path_on_datastore)
print(ds_data)

## Shred the dataset on the remote compute

The training data will need to be shredded to match the number of VMs and the worker count per VM, and then deployed to a mounted file system that the VM docker images have read/write access. A basic python script [data_shredding.py](./data_shredding.py) that can be used to read the raw data form datastore, then shred and deploy the training data to the same datastore for later training.

We will run the data shredding task in the remote compute.

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the entry script, and any additional files your training script depends on.

In [ ]:
import os

project_folder = './HPMLA'
os.makedirs(project_folder, exist_ok=True)

Copy the training script `data_shredding.py` into this project directory.

In [ ]:
import shutil
shutil.copy('data_shredding.py', project_folder)

### Create an experiment
Create an [experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed HPMLA tutorial. 

In [ ]:
from azureml.core import Experiment

experiment_name = 'HPMLA-exp'
experiment = Experiment(ws, name=experiment_name)

### Create an Estimator
The AML SDK's base Estimator enables you to easily submit custom scripts for both single-node and distributed runs. You should this generic estimator for training code using frameworks such as sklearn or HPMLA that don't have corresponding custom estimators. For more information on using the generic estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-ml-models).

In [ ]:
from azureml.train.estimator import *

script_params = {
    '--worker_count': 4,
    '--input_path': ds_data.path('rawdata').as_mount(),
    '--output_path': ds_data.path('data_shred').as_mount(),
    '--training_data_shred_count': 100,
    '--trainind_dataset_name': 'shred_data'
}

data_shred_estimator = Estimator(source_directory=project_folder,
                      compute_target=compute_target,
                      entry_script='data_shredding.py',
                      script_params=script_params,
                      custom_docker_base_image='batchaitraining/symsgd',
                      node_count=1,
                      use_gpu=False)

We would like to use a [pre-built Docker container](https://hub.docker.com/r/batchaitraining/symsgd/tags/). To do so, specify the name of the docker image to the argument `custom_docker_base_image`. You can only provide images available in public docker repositories such as Docker Hub using this argument.

Please find the [dockerfile](./dockerfile) of the custom image used in the job `batchaitraining/symsgd`

### Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [ ]:
data_shred_run = experiment.submit(data_shred_estimator)
print(data_shred_run.get_details())

### Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [ ]:
from azureml.train.widgets import RunDetails
RunDetails(data_shred_run).show()

Alternatively, you can block until the script has completed training before running more code.

In [ ]:
data_shred_run.wait_for_completion(show_output=True)

## Train Model on the remote compute
Now that we have the shredded data ready to go, let's run our distributed training job.Since currently AML does not support custom executable file to launch, we have prepared a python wrapper [`hpmla_launcher.py`](./hpmla_launcher.py) to launch HPMLA superSGD excutable

We first need to copy the launcher script into local project directory.

In [ ]:
shutil.copy('hpmla_launcher.py', project_folder)

### Create an Estimator
As introduced in earlier session, we need to define a new AML estimator for our distributed training. The training will use 2 nodes and start 2 workers on each node. OpenMPI comminication backend will be used.

In [ ]:
from azureml.train.estimator import *

script_params = {
    '--f': ds_data.path('data_shred/shred_data-').as_mount(),
    '--glDir':'./outputs',
    '--bd': ds_data.path('data_shred/shred_data-').as_mount()
}

train_estimator = Estimator(source_directory=project_folder,
                      compute_target=compute_target,
                      entry_script='hpmla_launcher.py',
                      script_params=script_params,
                      node_count=2,
                      process_count_per_node=2,
                      custom_docker_base_image='batchaitraining/symsgd',
                      distributed_backend='mpi',
                      use_gpu=False)

### Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [ ]:
train_run = experiment.submit(train_estimator)
print(train_run.get_details())

### Monitor your run

In [ ]:
from azureml.train.widgets import RunDetails
RunDetails(train_run).show()

In [ ]:
train_run.wait_for_completion(show_output=True)